<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/main/GTFS_penultimate_cleanup_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC note 250726-1715: I have some concerns about the anomaly placement, but I also haven't worked with a multi-day dataset previously, so I need to do some more testing.


#Load the Libraries

In [3]:
#Required Libraries
import pandas as pd
import numpy as np

# # Install if needed
# !pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

from datetime import datetime
import zipfile
import urllib.request

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

from folium import Map, FeatureGroup, CircleMarker, PolyLine, Marker, Icon, LayerControl
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import folium

!pip install psycopg2-binary pandas sqlalchemy

import psycopg2
from sqlalchemy import create_engine
import os
from google.colab import userdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.9 MB/s eta 0:00:00


In [4]:
# Database connection parameters
DB_NAME = "abq-transit-db"

# You'll need to store your database password/paramenters in Colab Secrets
DB_HOST = userdata.get('DB_HOST_2')
DB_PORT = userdata.get('DB_PORT')
DB_PASSWORD = userdata.get('DB_PASSWORD')
DB_USER = userdata.get("DB_USER")


# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

In [5]:
#Collect Static zip
def download_file_urllib(url, local_filename):
    """
    Downloads a file from a URL using urllib.request and saves it locally.
    """
    try:
        urllib.request.urlretrieve(url, local_filename)
        print(f"File downloaded successfully to: {local_filename}")
    except Exception as e:
        print(f"Error downloading file: {e}")

# Example usage:
file_url = "https://data.cabq.gov/transit/gtfs/google_transit.zip"  # Replace with your URL
output_filename = "google_transit.zip"
download_file_urllib(file_url, output_filename)

File downloaded successfully to: google_transit.zip


In [6]:
# Select Parameters for Anomaly Detection
# This section sets threshholds for classifying anomalous transit events.

# --- Use meters for user input ---
# The jump_thresh_m_widget creates a sliding bar for selecting how much distance there must be between
# two consecutive positions for a single vehicle to have gone 'unusually far' or 'jumped'.
jump_thresh_m_widget = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description='Jump Distance',
    layout=widgets.Layout(width='60%')
)

# Other anomaly widgets
# The disappear_thresh_widget creates a sliding bar for selecting how many consecutive seconds  must pass
# without a vehicle transmitting data before we consider that vehicle to have been gone 'unusually long'.
disappear_thresh_widget = widgets.IntSlider(value=300, min=60, max=1800, step=30, description='Time Gap')

# The min_jump_widget creates a sliding bar for selecting how many 'jumps' a vehicle must make before
# we consider the jumping to be 'unusual'.
# SUGGESTION: can we make this based off of jump points as a percentage of total recorded points a required number of consecutive jumps?
# As we use more data, any set value of total jumps will become more and more easily met.
min_jump_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Jumps')

# The speed_thresh_widget creates a sliding bar for selecting how fast a vihicke must be going for us
# to consider that vehicle as 'unusually fast'.
speed_thresh_widget = widgets.IntSlider(value=70, min=10, max=100, step=5, description='Speed')

# motion to depricate this widget
reversal_thresh_widget = widgets.IntSlider(value=120, min=60, max=180, step=5, description='Heading Δ°')

# --- Stuck vehicle detection ---
# The stuck_speed_widget creates a sliding bar for selecting how slow a vehicle must be going
# for us to consider it as effectively unmoving.
stuck_speed_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Stuck Speed',
    layout=widgets.Layout(width='60%')
)

# The stuck_window_widget creates a sliding bar for selecting how many consecutive Stuck Speed
# events a vehicle must have before it is considered to have been still for an unusually long period.
stuck_window_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Window Size',
    layout=widgets.Layout(width='60%')
)

# --- Repeated point tolerance ---
# motion to depricate this widget
repeat_tolerance_widget = widgets.FloatLogSlider(
    value=1e-5,
    base=10,
    min=-7,  # 1e-7
    max=-3,  # 1e-3
    step=0.1,
    description='Repeat Tolerance',
    layout=widgets.Layout(width='60%')
)

# --- Early appearance margin ---
# motion to depricate this widget
early_margin_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=600,
    step=10,
    description='Early Margin',
    layout=widgets.Layout(width='60%')
)

# --- Off-route buffer ---
# The offroute_buffer_widget creates a sliding bar for selecting how many meters
# a vehicle must be off of its expected route before we actually label it as off route
offroute_buffer_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description='Off-Route Buffer',
    layout=widgets.Layout(width='60%')
)

# Labels
jump_label = widgets.Label(value="(meters)")
disappear_label = widgets.Label(value="(seconds)")
min_jump_label = widgets.Label(value="(count)")
speed_label = widgets.Label(value="(mph)")
reversal_label = widgets.Label(value="(degrees of reversal)")
stuck_speed_label = widgets.Label(value="(mph)")
stuck_window_label = widgets.Label(value="(frames in rolling window)")
repeat_tolerance_label = widgets.Label(value="(decimal degrees)")
early_margin_label = widgets.Label(value="(seconds)")
offroute_buffer_label = widgets.Label(value="(meters)")

# Assemble UI layout
slider_widgets = widgets.VBox([
    widgets.HBox([jump_thresh_m_widget, jump_label]),
    widgets.HBox([disappear_thresh_widget, disappear_label]),
    widgets.HBox([min_jump_widget, min_jump_label]),
    widgets.HBox([speed_thresh_widget, speed_label]),
    widgets.HBox([reversal_thresh_widget, reversal_label]),
    widgets.HBox([stuck_speed_widget, stuck_speed_label]),
    widgets.HBox([stuck_window_widget, stuck_window_label]),
    widgets.HBox([repeat_tolerance_widget, repeat_tolerance_label]),
    widgets.HBox([early_margin_widget, early_margin_label]),
    widgets.HBox([offroute_buffer_widget, offroute_buffer_label]),
])

# Button and save logic
submit_button = widgets.Button(description="Save Parameters", button_style='primary')
anomaly_params = {}

def save_params(b):
    '''
    save_params displays widgets that allow you to set up and save anomaly detection threshholds.
    '''
    clear_output(wait=True)
    display(slider_widgets, submit_button)

    global anomaly_params
    jump_m = jump_thresh_m_widget.value
    jump_deg = jump_m / 111000  # Convert meters → degrees

    anomaly_params = {
        "JUMP_DISTANCE_THRESHOLD": jump_deg,
        "JUMP_DISTANCE_METERS": jump_m,
        "DISAPPEARANCE_TIME_THRESHOLD": disappear_thresh_widget.value,
        "MIN_JUMP_COUNT_PER_VEHICLE": min_jump_widget.value,
        "SPEED_LIMIT_MPH": speed_thresh_widget.value,
        "REVERSAL_HEADING_THRESHOLD": reversal_thresh_widget.value,
        "STUCK_SPEED_MPH": stuck_speed_widget.value,
        "STUCK_WINDOW_SIZE": stuck_window_widget.value,
        "REPEATED_COORD_TOLERANCE": repeat_tolerance_widget.value,
        "EARLY_APPEARANCE_MARGIN_SEC": early_margin_widget.value,
        "OFF_ROUTE_BUFFER_M": offroute_buffer_widget.value,
    }

    print("✅ Anomaly detection parameters set:")
    for k, v in anomaly_params.items():
        print(f"  {k}: {v}")

submit_button.on_click(save_params)

# Display interface
display(slider_widgets, submit_button)


Button(button_style='primary', description='Save Parameters', style=ButtonStyle())

✅ Anomaly detection parameters set:
  JUMP_DISTANCE_THRESHOLD: 0.0045045045045045045
  JUMP_DISTANCE_METERS: 500
  DISAPPEARANCE_TIME_THRESHOLD: 300
  MIN_JUMP_COUNT_PER_VEHICLE: 2
  SPEED_LIMIT_MPH: 70
  REVERSAL_HEADING_THRESHOLD: 120
  STUCK_SPEED_MPH: 1.0
  STUCK_WINDOW_SIZE: 4
  REPEATED_COORD_TOLERANCE: 1e-05
  EARLY_APPEARANCE_MARGIN_SEC: 30
  OFF_ROUTE_BUFFER_M: 50


#LOAD DATA (DB or Pickle)

In [ ]:
# This cell should be used when loading pickle files as opposed to using the database
#import pickle

# Load a pickle file of a dataframe which contains our historic transit data.
#with open("captures.p", "rb") as f:
#    captured_data = pickle.load(f)


In [7]:
with engine.connect() as connection:
    captured_data = pd.read_sql_query(
        '''
        SELECT
          id, snapshot_id, msg_time, timestamp_collected, vehicle_id, ST_X("vehicle_snapshots"."location") AS longitude, ST_Y("vehicle_snapshots"."location") AS latitude, heading, speed_mph,
          route_short_name, trip_id, next_stop_id, next_stop_id, next_stop_name,
          next_stop_sched_time
        FROM
          vehicle_snapshots
        WHERE
          DATE("timestamp_collected") = (CURRENT_DATE - INTERVAL '1 DAY')
        ORDER BY
          timestamp_collected DESC;
        ''', connection)

captured_data.head()

,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_id,next_stop_name,next_stop_sched_time
0,4865959,793,11:08:57,2025-08-05 23:59:56.121302+00:00,2323,-106.733934,35.089727,155.0,0.0,Off Duty,0,0,0,No Data,11:08:57
1,4865960,793,19:12:39,2025-08-05 23:59:56.121302+00:00,2324,-106.734521,35.090541,124.0,0.0,Off Duty,0,0,0,No Data,19:12:39
2,4865961,793,17:48:08,2025-08-05 23:59:56.121302+00:00,2325,-106.732952,35.089238,88.0,0.0,Off Duty,0,0,0,No Data,17:48:08
3,4865962,793,17:59:19,2025-08-05 23:59:56.121302+00:00,2326,-106.586329,35.073055,270.0,21.0,Off Duty,0,0,0,No Data,17:59:19
4,4865963,793,17:59:26,2025-08-05 23:59:56.121302+00:00,2327,-106.733140,35.089337,91.0,0.0,Off Duty,0,0,0,No Data,17:59:26


##Data Validation

In [8]:
cd_df = captured_data
cd_df.size

14542680

In [10]:
# Convert timestamp_collected & msg_time to datetime
cd_df['timestamp_collected'] = pd.to_datetime(cd_df['timestamp_collected'], errors='coerce')
cd_df['msg_time'] = pd.to_datetime(cd_df['msg_time'], errors='coerce')

# 1. Shape and Column Overview
print("Shape:", cd_df.shape)
print("Columns:", cd_df.columns.tolist())

# 2. Null/Missing Value Counts
print("\nMissing Values:\n", cd_df.isnull().sum())

# 3. Duplicate Detection
duplicates = cd_df.duplicated().sum()
print("\nDuplicate rows:", duplicates)

# 4. Timestamp Range & msg_time range
print("\nTimestamp Range:")
print("Min:", cd_df['timestamp_collected'].min())
print("Max:", cd_df['timestamp_collected'].max())
print("Duration:", cd_df['timestamp_collected'].max() - cd_df['timestamp_collected'].min())

print("\nmsg_time:")
print("Min:", cd_df['msg_time'].min())
print("Max:", cd_df['msg_time'].max())
print("Range:", cd_df['msg_time'].max() - cd_df['msg_time'].min())

# 5. Latitude/Longitude Range
print("\nLatitude Range:", cd_df['latitude'].min(), "to", cd_df['latitude'].max())
print("Longitude Range:", cd_df['longitude'].min(), "to", cd_df['longitude'].max())

# 6. Speed Summary
print("\nSpeed Summary:\n", cd_df['speed_mph'].describe())


/tmp/ipython-input-3544357411.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cd_df['msg_time'] = pd.to_datetime(cd_df['msg_time'], errors='coerce')


Shape: (969512, 15)
Columns: ['id', 'snapshot_id', 'msg_time', 'timestamp_collected', 'vehicle_id', 'longitude', 'latitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time']

Missing Values:
 id                      0
snapshot_id             0
msg_time                0
timestamp_collected     0
vehicle_id              0
longitude               0
latitude                0
heading                 0
speed_mph               0
route_short_name        0
trip_id                 0
next_stop_id            0
next_stop_id            0
next_stop_name          0
next_stop_sched_time    0
dtype: int64

Duplicate rows: 0

Timestamp Range:
Min: 2025-08-05 00:00:05.021925+00:00
Max: 2025-08-05 23:59:56.121302+00:00
Duration: 0 days 23:59:51.099377

msg_time:
Min: 2025-08-06 00:00:11
Max: 2025-08-06 23:59:39
Range: 0 days 23:59:28

Latitude Range: 0.0 to 39.1386265
Longitude Range: -106.79177 to 0.0

Speed Summary:
 count   

In [11]:
# Select rows where 'longitude' and 'latitude' have the same value
rows_with_same_values = cd_df[cd_df['longitude'] == cd_df['latitude']]
rows_without_same_values = cd_df[cd_df['longitude'] != cd_df['latitude']]

# Print the resulting DataFrame
print(rows_with_same_values)

             id  snapshot_id            msg_time  \
13      4865972          793 2025-08-06 20:00:59   
459     4865627          792 2025-08-06 20:00:59   
726     4865282          791 2025-08-06 20:00:59   
1071    4864937          790 2025-08-06 20:00:59   
1416    4864592          789 2025-08-06 20:00:59   
...         ...          ...                 ...   
967827  3898181         8601 2025-08-06 20:00:59   
968171  3897837         8600 2025-08-06 20:00:59   
968515  3897493         8599 2025-08-06 20:00:59   
968859  3897149         8598 2025-08-06 20:00:59   
969325  3896805         8597 2025-08-06 20:00:59   

                    timestamp_collected vehicle_id  longitude  latitude  \
13     2025-08-05 23:59:56.121302+00:00       4001        0.0       0.0   
459    2025-08-05 23:59:25.521522+00:00       4001        0.0       0.0   
726    2025-08-05 23:58:55.222128+00:00       4001        0.0       0.0   
1071   2025-08-05 23:58:25.621610+00:00       4001        0.0       0.0   


####*.v8.2 update
We're removing the off-duty rows where Lat = Long = 0

In [12]:
cd_df_copy = cd_df.copy()
cd_df_copy.shape

(969512, 15)

In [13]:
cd_df = cd_df[cd_df['longitude'] != cd_df['latitude']]
cd_df.shape

(966701, 15)

#Convert code for snapshot to use Pickle

In [14]:
def filter_cd_df(df, start_time=None, end_time=None, routes=None, vehicles=None, bounds=None):
    '''
    Filters a dataframe(df) such that:
    - the "timestamp_collected" column's entries are greater than or equal to the start_time
    - the "timestamp_collected" column's entries are less than or equal to the end_time
    - the "route_short_name" column's entries are in routes
    - the "vehicle_id" column's entries are in vehicles

    Returns the filtered dataframe.
    '''
    df_filtered = df.copy()

    if start_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] >= pd.to_datetime(start_time)]
    if end_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] <= pd.to_datetime(end_time)]
    if routes:
        df_filtered = df_filtered[df_filtered['route_short_name'].isin(routes)]
    if vehicles:
        df_filtered = df_filtered[df_filtered['vehicle_id'].isin(vehicles)]
    if bounds:
        lat_min, lat_max, lon_min, lon_max = bounds
        df_filtered = df_filtered[
            (df_filtered['latitude'] >= lat_min) & (df_filtered['latitude'] <= lat_max) &
            (df_filtered['longitude'] >= lon_min) & (df_filtered['longitude'] <= lon_max)
        ]

    return df_filtered


In [15]:
# --- Widgets ---
# route_widget lets you select which routes you'd like to like to plot in folium from a menu.
route_widget = widgets.SelectMultiple(
    options=sorted(cd_df['route_short_name'].unique()),
    description='Routes',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# vehicle_widget lets you select which vehicles' data you'd like to plot in folium from a menu.
vehicle_widget = widgets.SelectMultiple(
    options=sorted(cd_df['vehicle_id'].unique()),
    description='Vehicles',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# start_widget lets you select the earliest date you'd like to use data from.
start_widget = widgets.DatePicker(
    description='Start Date',
    value=cd_df['timestamp_collected'].min().date()
)
# end_widget lets you select the latest date you'd like to use data from.
end_widget = widgets.DatePicker(
    description='End Date',
    value=cd_df['timestamp_collected'].max().date()
)

# hour_slider creates a set of sliding bars for selecting the earliest and latest
# point of the day that you'd like to use data from.
hour_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Hour Range',
    layout=widgets.Layout(width='60%')
)

# Makes a button, that will later be used to apply filters.
filter_button = widgets.Button(description='Apply Filters', button_style='primary')

# --- Global to store result ---
cd_df_filtered = pd.DataFrame()

# --- Filtering callback ---
def apply_filters(b):
    '''
    Uses previously defined widgets to filter a dataframe of historical transit data
    by rout_short_name, vehicle_id, and timestamp collected.
    Prints the number of filtered rows and displays the first 10 rows of the filtered dataframe.
    '''
    global cd_df_filtered
    clear_output(wait=True)
    display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)

    df = cd_df.copy()

    # Apply filters
    if route_widget.value:
        df = df[df['route_short_name'].isin(route_widget.value)]
    if vehicle_widget.value:
        df = df[df['vehicle_id'].isin(vehicle_widget.value)]

    start_dt = pd.to_datetime(start_widget.value).tz_localize('UTC')
    end_dt = (pd.to_datetime(end_widget.value) + pd.Timedelta(days=1)).tz_localize('UTC')
    df = df[(df['timestamp_collected'] >= start_dt) &
            (df['timestamp_collected'] < end_dt)]

    hr_start, hr_end = hour_slider.value
    df = df[(df['timestamp_collected'].dt.hour >= hr_start) &
            (df['timestamp_collected'].dt.hour <= hr_end)]

    # Store result globally
    cd_df_filtered = df

    print(f"✅ Filtered {len(df)} rows.")
    display(df.head(10))

# Displays a filter button that calls the 'apply_filters' function
filter_button.on_click(apply_filters)

# --- Display UI ---
display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)


SelectMultiple(description='Routes', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

SelectMultiple(description='Vehicles', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18…

DatePicker(value=datetime.date(2025, 8, 5), description='Start Date')

DatePicker(value=datetime.date(2025, 8, 5), description='End Date')

IntRangeSlider(value=(0, 23), description='Hour Range', layout=Layout(width='60%'), max=23)

Button(button_style='primary', description='Apply Filters', style=ButtonStyle())

✅ Filtered 103257 rows.


,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_id,next_stop_name,next_stop_sched_time
152,4865705,793,2025-08-06 17:59:30,2025-08-05 23:59:56.121302+00:00,1903,-106.723280,35.078312,346.0,28.0,766,602958,704230287,704230287,CUTC Bay B,18:10:00
153,4865706,793,2025-08-06 17:59:29,2025-08-05 23:59:56.121302+00:00,1906,-106.627966,35.081583,277.0,30.0,766,602720,704230852,704230852,ART - Presbyterian Station,17:56:16
154,4865707,793,2025-08-06 17:59:30,2025-08-05 23:59:56.121302+00:00,1909,-106.648098,35.083560,10.0,0.0,766,602679,704230872,704230872,ART - EDo Station,17:56:22
155,4865708,793,2025-08-06 17:59:28,2025-08-05 23:59:56.121302+00:00,1913,-106.585826,35.077770,98.0,21.0,766,602678,704230973,704230973,ART - San Pedro Station,17:49:31
156,4865709,793,2025-08-06 17:59:23,2025-08-05 23:59:56.121302+00:00,1918,-106.568897,35.097604,180.0,22.0,766,602721,704230154,704230154,Louisiana @ Lomas,18:00:20
157,4865710,793,2025-08-06 17:59:23,2025-08-05 23:59:56.121302+00:00,1902,-106.678807,35.091690,27.0,28.0,777,602957,704230894,704230894,ART - BioPark Station,17:56:00
158,4865711,793,2025-08-06 17:59:29,2025-08-05 23:59:56.121302+00:00,1908,-106.669777,35.094242,319.0,20.0,777,602994,704230895,704230895,ART - BioPark Station,17:58:00
159,4865712,793,2025-08-06 17:59:25,2025-08-05 23:59:56.121302+00:00,1912,-106.615555,35.081051,92.0,28.0,777,602956,704231125,704231125,ART - Nob Hill Station,17:54:00
160,4865713,793,2025-08-06 17:59:19,2025-08-05 23:59:56.121302+00:00,1915,-106.499279,35.065976,272.0,0.0,777,602996,704231213,704231213,Wenonah @ Tramway (ART),18:04:00
161,4865714,793,2025-08-06 17:59:30,2025-08-05 23:59:56.121302+00:00,1920,-106.723460,35.078548,165.0,15.0,777,602680,4359,4359,CUTC Bay A,17:58:00


In [16]:
#Pickle replacement code
df_new = cd_df_filtered


In [17]:
#✅ Step 2a: Trip ID Validity
# Counts and displays the total rows and number of rows with invalid trip ids.
# Calculates and displays the percentage of rows with valid trip ids.
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid trip_ids: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


Total rows: 103257
Invalid trip_ids: 10075
Percent valid trip_ids: 90.24%


In [18]:
#*.v8.2 new calc for finding mostly Off Duty
invalid_route_ids = df_new["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 0
Percent valid route_ids: 100.00%


In [19]:
invalid_ids = ["0", "Undetermined", "nan", "", "None", "Off Duty"]
df_clean_test = df_new[~df_new["trip_id"].isin(invalid_ids) & ~df_new["route_short_name"].isin(invalid_ids)].copy()
df_clean_test.shape

(93182, 15)

In [20]:
#✅ Step 2b: Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean_trips = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()
df_clean_trips.shape

(93182, 15)

In [21]:
#So after all that we can see that all of the rows dropped by df_clean_trips cover those dropped by df_clean_test (both invalid trips and routes)
invalid_route_ids = df_clean_trips["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 0
Percent valid route_ids: 100.00%


In [22]:
df_clean = df_clean_test

####Future note
We can skip several of the above cells, as dropping the invalid trip_ids also drops the invalid route_short_names

#Load the Static data

In [23]:
#✅ Step 1b: Reload Static GTFS and Normalize It (ensure data types are consistent)

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [24]:
#✅ Step 3b: Merge with trips.txt to Get route_id
# Take all of df_clean and add on data from trips if the trip_id matches across both dataframes.
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [25]:
#Let's avoid the route_..._x and route_..._y situation on the merge:
#✅ Step 4b: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Assume 'df_with_trips' and 'routes' both contain 'route_id' and 'route_short_name'

# Step 1: Check 1:1 match on overlapping column
merged_check = df_with_trips[['route_id', 'route_short_name']].merge(
    routes[['route_id', 'route_short_name']],
    on='route_id',
    how='inner',
    suffixes=('_df_with_trips', '_routes')
)


In [1]:
# ##Note: This outputting an error is working as intended; it is showing the mismatches, just as an error instead of a standard output
# # Step 2: Find any mismatches
# mismatch = merged_check[merged_check['route_short_name_df_with_trips'] != merged_check['route_short_name_routes']]

# if not mismatch.empty:
#     raise ValueError(f"Mismatch found in route_short_name across dataframes:\n{mismatch}")

In [2]:
# This will display the mismatches by row, if we want to look at that again
# #Okay... so there are some mismatches between the data and the static routes info; let's see what they are
# mismatch_summary = (
#     mismatch[['route_id', 'route_short_name_df_with_trips', 'route_short_name_routes']]
#     .drop_duplicates()
#     .sort_values('route_id')
# )
# display(mismatch_summary)


In [26]:
# Rename to preserve both versions of route_short_name
routes_renamed = routes.rename(columns={'route_short_name': 'route_short_name_static'})

# Merge with df_with_trips, keeping the RT version as primary
merged = df_with_trips.merge(routes_renamed, on='route_id', how='left')


In [27]:
df_full = merged

In [28]:
#✅ Step 5b: Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)

In [29]:
#✅ Step 6b: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


data_quality
valid    93182
Name: count, dtype: int64


#Clean the RT data

In [30]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 84


#Anomly Detection

In [31]:
# 🔹 1. Detect Jumps and Gaps – WITHIN trip_id
def detect_jumps_and_gaps(df, params):
    """
     Detects jump or gap anomalies based on distance (meters) and time (seconds)
     thresholds pulled from the anomaly_params dictionary.

    Parameters:
        df (pd.DataFrame): Filtered DataFrame for a single vehicle, sorted by timestamp.
                           Must include 'trip_id', 'timestamp', 'latitude', 'longitude'.
        params (dict): Anomaly detection thresholds from widget interface.

    Returns:
        List[dict]: Detected jump or gap anomalies.
    """
    distance_threshold = params["JUMP_DISTANCE_METERS"]           # meters
    time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # seconds

    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]

        # 💡 Trip boundary check — skip if different trip
        if row_prev["trip_id"] != row_curr["trip_id"]:
            continue

        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "trip_id": row_curr["trip_id"],  # optional but useful for tracing
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })

    return anomalies


In [32]:
# 🔹 2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, params):
    """
    Detects stuck vehicles using a rolling window of speed and spatial consistency,
    segmented by trip_id to avoid cross-trip contamination.

    Parameters:
        df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
        params (dict): Widget-defined anomaly thresholds.

    Returns:
        pd.DataFrame: Flagged stuck vehicle anomalies.
    """
    speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # mph
    window = params.get("STUCK_WINDOW_SIZE", 4)         # frames

    flagged_list = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute rolling conditions
        speed_ok = group["speed_mph"].rolling(window).mean() < speed_thresh
        lat_ok = group["latitude"].diff().abs().rolling(window).mean() < 0.0001
        lon_ok = group["longitude"].diff().abs().rolling(window).mean() < 0.0001

        stuck_flags = speed_ok & lat_ok & lon_ok

        flagged = group[stuck_flags.fillna(False)].copy()
        if not flagged.empty:
            flagged["anomaly_type"] = "stuck_vehicle"
            flagged_list.append(flagged)

    return pd.concat(flagged_list, ignore_index=True) if flagged_list else pd.DataFrame(columns=df.columns.tolist() + ["anomaly_type"])


In [33]:
# 🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, params):
    """
    Detects movement segments where computed speed exceeds threshold,
    segmented by trip_id to avoid cross-trip errors.

    Parameters:
        df (pd.DataFrame): Must contain timestamp, lat/lon, trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Flagged rows with 'impossible_speed'
    """
    speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934
    output_rows = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        for i in range(1, len(group)):
            row_prev, row_curr = group.iloc[i - 1], group.iloc[i]
            time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
            if time_diff <= 0:
                continue

            distance = geodesic(
                (row_prev["latitude"], row_prev["longitude"]),
                (row_curr["latitude"], row_curr["longitude"])
            ).meters

            speed_kph = (distance / time_diff) * 3.6

            if speed_kph > speed_limit_kph:
                output_rows.append({
                    "vehicle_id": row_curr["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp_curr": row_curr["timestamp"],
                    "computed_speed_kph": speed_kph,
                    "distance_m": distance,
                    "anomaly_type": "impossible_speed"
                })

    return pd.DataFrame(output_rows)


In [34]:
# 🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, params):
    """
    Detects heading reversals (backtracking) within trip boundaries only,
    with wraparound-safe angle diffing and traceability.

    Parameters:
        df (pd.DataFrame): Filtered per-vehicle data with heading + trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'backtracking'
    """
    reversal_thresh = params.get("REVERSAL_HEADING_THRESHOLD", 120)
    output_frames = []

    if "heading" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute heading difference with wraparound correction
        group["heading_prev"] = group["heading"].shift()
        group["heading_diff"] = (group["heading"] - group["heading_prev"]).abs()
        group["heading_diff"] = group["heading_diff"].apply(
            lambda x: min(x, 360 - x) if pd.notna(x) else x
        )

        # Identify backtracking
        backtrack_flags = group["heading_diff"].between(reversal_thresh, 200)

        flagged = group[backtrack_flags.fillna(False)].copy()
        flagged["anomaly_type"] = "backtracking"
        flagged["timestamp_prev"] = group["timestamp"].shift()

        output_frames.append(flagged)

    return pd.concat(output_frames, ignore_index=True) if output_frames else pd.DataFrame()



In [35]:
# 🔹 5. Detect Repeated Points
def detect_repeated_points(df, params):
    """
    Detects repeated GPS coordinates (vehicle not moving), scoped to each trip.

    Parameters:
        df (pd.DataFrame): Per-vehicle GPS snapshot data
        params (dict): UI-provided anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'repeated_points'
    """
    tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))
    output = []

    if "latitude" not in df.columns or "longitude" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()
        lat_repeat = group["latitude"].diff().abs() < tolerance
        lon_repeat = group["longitude"].diff().abs() < tolerance
        repeated = lat_repeat & lon_repeat

        flagged = group[repeated.fillna(False)].copy()
        flagged["anomaly_type"] = "repeated_points"
        output.append(flagged)

    return pd.concat(output, ignore_index=True) if output else pd.DataFrame()


In [36]:
# # 🔹 6. Detect Disappearance Without Return
# def detect_disappeared(df, snapshot_end_time, params):
#     """
#     Detects disappearance per trip_id — if the trip ends and never resumes.

#     Parameters:
#         df (pd.DataFrame): Data for a single vehicle
#         snapshot_end_time (datetime): End of snapshot
#         params (dict): Anomaly config

#     Returns:
#         pd.DataFrame: One row per disappeared trip, if any
#     """
#     disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]
#     output = []

#     for trip_id, group in df.groupby("trip_id"):
#         last_seen = group["timestamp"].max()
#         time_gap_sec = (snapshot_end_time - last_seen).total_seconds()

#         if time_gap_sec > disappearance_thresh:
#             output.append({
#                 "vehicle_id": group["vehicle_id"].iloc[0],
#                 "trip_id": trip_id,
#                 "last_seen": last_seen,
#                 "anomaly_type": "disappearance"
#             })

#     return pd.DataFrame(output) if output else pd.DataFrame()


In [37]:
# # 🔹 7. Detect Early Appearance
# def detect_early_appearance(df, snapshot_start_time, params):
#     """
#     Detects vehicles that appear too early (likely pre-start ghost data).

#     Parameters:
#         df (pd.DataFrame): Data for a single vehicle
#         snapshot_start_time (datetime): Start of capture window
#         params (dict): Anomaly detection thresholds from widget

#     Returns:
#         pd.DataFrame: Single-row anomaly, or empty DataFrame
#     """
#     margin_seconds = params.get("EARLY_APPEARANCE_MARGIN_SEC", 30)
#     first_seen = df["timestamp"].min()
#     delta = (first_seen - snapshot_start_time).total_seconds()

#     if delta < margin_seconds:
#         return pd.DataFrame([{
#             "vehicle_id": df["vehicle_id"].iloc[0],
#             "first_seen": first_seen,
#             "anomaly_type": "early_appearance"
#         }])
#     return pd.DataFrame()


In [38]:
# 🔹 8. Detect Off-Route Movement
def detect_off_route(df_vehicle, shape_lines, params):
    """
    Detect off-route anomalies using trip-level shape validation.

    Parameters:
        df_vehicle (pd.DataFrame): All rows for a single vehicle.
        shape_lines (dict): shape_id → LineString
        params (dict): Dictionary with OFF_ROUTE_BUFFER_M

    Returns:
        pd.DataFrame: Flagged off-route points
    """
    buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
    records = []

    for trip_id, trip_df in df_vehicle.groupby("trip_id"):
        # Check if all rows share the same shape_id
        shape_ids = trip_df["shape_id"].dropna().unique()

        if len(shape_ids) != 1:
            # Ambiguous shape_id — skip this trip to avoid misflagging
            continue

        shape_id = str(shape_ids[0])
        if shape_id not in shape_lines:
            continue

        route_line = shape_lines[shape_id]

        for _, row in trip_df.iterrows():
            vehicle_point = Point(row["longitude"], row["latitude"])
            projected = route_line.interpolate(route_line.project(vehicle_point))
            dist_m = geodesic(
                (row["latitude"], row["longitude"]),
                (projected.y, projected.x)
            ).meters

            if dist_m > buffer_m:
                records.append({
                    "vehicle_id": row["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp": row["timestamp"],
                    "route_short_name": row.get("route_short_name"),
                    "distance_from_route_m": dist_m,
                    "latitude": row["latitude"],
                    "longitude": row["longitude"],
                    "shape_id": shape_id,
                    "anomaly_type": "off_route"
                })

    return pd.DataFrame(records)

# Note: ✅ Key Protections Added
# Fix	Explanation
# groupby("trip_id")	Ensures anomalies only detected within a single trip context
# unique shape_id check	Filters out data where shape mapping is ambiguous
# skip trip if shape_id missing or mismatched	Prevents false positives from misassigned shapes
# Includes trip_id in output	Improves downstream traceability and debugging

In [87]:
# Define Anomalies to Detect
detector_options = [
    ("Jumps & Gaps", "jumpgap"),
    ("Stuck Vehicles", "stuck"),
    ("Impossible Speeds", "speed"),
    ("Backtracking", "backtrack"),
    ("Repeated Points", "repeated"),
    # ("Disappearance", "disappear"),
    # ("Early Appearance", "early"),
    ("Off-Route", "offroute")
]

# Create selector widget
detector_selector = widgets.SelectMultiple(
    options=detector_options,
    value=[k for _, k in detector_options],  # default: all selected
    description="Detectors:",
    layout=widgets.Layout(width="60%")
)

# Create button
confirm_button = widgets.Button(
    description="Confirm Selection",
    button_style='primary',
    icon='check'
)

# Output area
selection_output = widgets.Output()

# Define callback
def on_confirm_clicked(b):
    with selection_output:
        clear_output()
        selected = list(detector_selector.value)
        print("✅ Selected anomaly detectors:")
        for detector in selected:
            print(f" - {detector}")
        # Optionally set a global variable for later use:
        global selected_detectors
        selected_detectors = selected

# Link button to callback
confirm_button.on_click(on_confirm_clicked)

# Display widgets
display(widgets.VBox([detector_selector, confirm_button, selection_output]))


In [65]:
df_valid.head()

,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,...,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp,route_id_trips,route_short_name_trips
0,3943664,8734,2025-08-06 19:07:44,2025-08-05 01:08:36.429770+00:00,1901,-106.678204,35.093077,199.0,0.0,777,...,ART Green Line,NaN,3,NaN,00FF00,000000,valid,2025-08-05 01:08:36.429770+00:00,4331,777
1,3944008,8735,2025-08-06 19:07:44,2025-08-05 01:09:05.821371+00:00,1901,-106.678204,35.093077,199.0,0.0,777,...,ART Green Line,NaN,3,NaN,00FF00,000000,valid,2025-08-05 01:09:05.821371+00:00,4331,777
2,3944352,8736,2025-08-06 19:09:20,2025-08-05 01:09:36.220869+00:00,1901,-106.681221,35.088859,216.0,30.0,777,...,ART Green Line,NaN,3,NaN,00FF00,000000,valid,2025-08-05 01:09:36.220869+00:00,4331,777
3,3944696,8737,2025-08-06 19:09:20,2025-08-05 01:10:06.720862+00:00,1901,-106.681221,35.088859,216.0,30.0,777,...,ART Green Line,NaN,3,NaN,00FF00,000000,valid,2025-08-05 01:10:06.720862+00:00,4331,777
4,3945040,8738,2025-08-06 19:09:20,2025-08-05 01:10:36.521790+00:00,1901,-106.681221,35.088859,216.0,30.0,777,...,ART Green Line,NaN,3,NaN,00FF00,000000,valid,2025-08-05 01:10:36.521790+00:00,4331,777


In [88]:
# Selected detectors from UI
selected_detectors = list(detector_selector.value)

# Reset result containers
jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
# disappear_records = []
# early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

# Run detectors conditionally
for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    if "jumpgap" in selected_detectors:
        jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

    if "stuck" in selected_detectors:
        stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
        if not stuck.empty:
            stuck_records.append(stuck)

    if "speed" in selected_detectors:
        speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
        if not speed.empty:
            speed_records.append(speed)

    if "backtrack" in selected_detectors:
        backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
        if not backtrack.empty:
            backtrack_records.append(backtrack)

    if "repeated" in selected_detectors:
        repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
        if not repeat.empty:
            repeated_records.append(repeat)

    # if "disappear" in selected_detectors:
    #     disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
    #     if not disappear.empty:
    #         disappear_records.append(disappear)

    # if "early" in selected_detectors:
    #     early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
    #     if not early.empty:
    #         early_records.append(early)

    if "offroute" in selected_detectors:
        offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
        if not offroute.empty:
            offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
# df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
# df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# Step 4: Combine all selected anomalies
anomaly_frames = []
if "jumpgap" in selected_detectors:   anomaly_frames.append(df_anomalies_jumpgap)
if "stuck" in selected_detectors:     anomaly_frames.append(df_anomalies_stuck)
if "speed" in selected_detectors:     anomaly_frames.append(df_anomalies_speed)
if "backtrack" in selected_detectors: anomaly_frames.append(df_anomalies_backtrack)
if "repeated" in selected_detectors:  anomaly_frames.append(df_anomalies_repeated)
# if "disappear" in selected_detectors: anomaly_frames.append(df_anomalies_disappear)
# if "early" in selected_detectors:     anomaly_frames.append(df_anomalies_early)
if "offroute" in selected_detectors:  anomaly_frames.append(df_anomalies_offroute)

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]

#New *v9.2 code to try to trim off duplicate columns
# Step 5: Clean columns, concatenate frames, and inspect duplicates
anomaly_frames_cleaned = []
for i, df in enumerate(anomaly_frames):
    if not df.columns.is_unique:
        dup_cols = df.columns[df.columns.duplicated()].tolist()
        print(f"⚠️ Cleaning duplicate columns in anomaly_frames[{i}]: {dup_cols}")
        df = df.loc[:, ~df.columns.duplicated()]  # Drop duplicate columns, keep first
    anomaly_frames_cleaned.append(df)

# Concatenate all cleaned frames
df_anomalies_full = pd.concat(anomaly_frames_cleaned, ignore_index=True) if anomaly_frames_cleaned else pd.DataFrame()

# Inspect and drop row duplicates
if not df_anomalies_full.empty:
    df_anomalies_duplicates = df_anomalies_full[df_anomalies_full.duplicated(keep='first')]
    print("🔎 Duplicate rows to be dropped:", len(df_anomalies_duplicates))
    display(df_anomalies_duplicates.head())
    df_anomalies_full.drop_duplicates(inplace=True)
else:
    df_anomalies_duplicates = pd.DataFrame()  # Fallback if no data

#Previous code before the switch above
#df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True) if anomaly_frames else pd.DataFrame()

# Report
print("Anomaly count (pre-dedup):", len(df_anomalies_full))
df_anomalies_full.drop_duplicates(inplace=True)
print("Anomaly count (post-dedup):", len(df_anomalies_full))

print("✅ Unified anomaly count:", len(df_anomalies_full))
if not df_anomalies_full.empty:
    display(df_anomalies_full["anomaly_type"].value_counts())
else:
    print("⚠️ No anomalies detected.")


🔎 Duplicate rows to be dropped: 0


,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,timestamp,route_short_name,distance_from_route_m,latitude,longitude,shape_id


Anomaly count (pre-dedup): 19398
Anomaly count (post-dedup): 19398
✅ Unified anomaly count: 19398


,count
anomaly_type,
jump_or_gap,13873
off_route,5525


In [89]:
# Ensure shape_id is a column (not index)
shape_route_map = shape_route_map.reset_index()

# Ensure both shape_id columns are string type before merge
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)
shape_route_map["shape_id"] = shape_route_map["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# # Patch missing route names
# df_anomalies_offroute["route_short_name"] = (
#     df_anomalies_offroute["route_short_name"]
#     .fillna(df_anomalies_offroute["route_short_name_from_map"])
# )

# # Clean up
# df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [90]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

if missing_routes > 0:
    print("Sample of anomalies with missing route_short_name:")
    display(df_anomalies_offroute[df_anomalies_offroute["route_short_name"].isnull()].head())


Remaining anomalies with missing route_short_name: 0


In [91]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 84


#Summary Stats by Route/Anomaly

In [92]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

In [93]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

# Ensure trip_id types match
df_valid["trip_id"] = df_valid["trip_id"].astype(str)
trip_to_route_lookup["trip_id"] = trip_to_route_lookup["trip_id"].astype(str)

#I believe this may have been resolved by splitting route_short_name into x and y
# # Drop route_short_name from df_valid if it exists to avoid merge collision
# if "route_short_name" in df_valid.columns:
#     df_valid = df_valid.drop(columns=["route_short_name"])

trip_to_route_lookup["route_id_trips"] = trip_to_route_lookup["route_id"].copy()
trip_to_route_lookup["route_short_name_trips"] = trip_to_route_lookup["route_short_name"].copy()


# Merge to patch in route_short_name
df_valid = df_valid.merge(trip_to_route_lookup[["trip_id", "route_id_trips", "route_short_name_trips"]], on="trip_id", how="left")


In [94]:
#Step 1: Build a lookup table
route_lookup = df_valid[["vehicle_id", "timestamp", "route_short_name"]].copy()
route_lookup["timestamp"] = pd.to_datetime(route_lookup["timestamp"], utc=True)


In [95]:
#Create a backup so we don't have to keep rerunning everything
df_anomalies_full_copy = df_anomalies_full.copy()

# #Rebuild from scratch:

# df_anomalies_full_backup = pd.concat([
    # df_anomalies_jumpgap,
    # df_anomalies_stuck,
    # df_anomalies_speed,
    # df_anomalies_backtrack,
    # df_anomalies_repeated,
    # df_anomalies_disappear,
    # df_anomalies_early,
    # df_anomalies_offroute
# ], ignore_index=True)

In [ ]:
# df_anomalies_full = df_anomalies_full_copy

In [97]:
#We had 'timestamp_collected' vanish, so we are creating an improved version
df_anomalies_full['timestamp_collected'] = df_anomalies_full['timestamp_curr'].combine_first(
    df_anomalies_full['timestamp']
)


In [98]:
df_anomalies_full

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,timestamp,route_short_name,distance_from_route_m,latitude,longitude,shape_id,timestamp_collected
0,1901,602997,2025-08-05 01:09:05.821371+00:00,2025-08-05 01:09:36.220869+00:00,30.399498,542.913657,False,True,jump_or_gap,NaT,NaN,NaN,NaN,NaN,NaN,2025-08-05 01:09:36.220869+00:00
1,1901,602997,2025-08-05 01:10:36.521790+00:00,2025-08-05 01:11:06.521102+00:00,29.999312,504.540157,False,True,jump_or_gap,NaT,NaN,NaN,NaN,NaN,NaN,2025-08-05 01:11:06.521102+00:00
2,1901,602997,2025-08-05 01:13:37.021242+00:00,2025-08-05 01:14:06.021052+00:00,28.999810,884.045381,False,True,jump_or_gap,NaT,NaN,NaN,NaN,NaN,NaN,2025-08-05 01:14:06.021052+00:00
3,1901,602997,2025-08-05 01:27:07.121822+00:00,2025-08-05 01:27:37.521216+00:00,30.399394,2603.720224,False,True,jump_or_gap,NaT,NaN,NaN,NaN,NaN,NaN,2025-08-05 01:27:37.521216+00:00
4,1901,602997,2025-08-05 02:08:07.521657+00:00,2025-08-05 02:08:37.722142+00:00,30.200485,12434.867275,False,True,jump_or_gap,NaT,NaN,NaN,NaN,NaN,NaN,2025-08-05 02:08:37.722142+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,983,602061,NaT,NaT,NaN,NaN,NaN,NaN,off_route,2025-08-05 01:28:07.622588+00:00,53,6696.960959,35.095734,-106.730025,17204,2025-08-05 01:28:07.622588+00:00
19394,983,602061,NaT,NaT,NaN,NaN,NaN,NaN,off_route,2025-08-05 01:28:37.422608+00:00,53,6696.960959,35.095734,-106.730025,17204,2025-08-05 01:28:37.422608+00:00
19395,983,602061,NaT,NaT,NaN,NaN,NaN,NaN,off_route,2025-08-05 01:29:06.529113+00:00,53,5968.797440,35.086141,-106.729582,17204,2025-08-05 01:29:06.529113+00:00
19396,983,602061,NaT,NaT,NaN,NaN,NaN,NaN,off_route,2025-08-05 01:29:37.022165+00:00,53,5968.797440,35.086141,-106.729582,17204,2025-08-05 01:29:37.022165+00:00


In [99]:
#Let's redo the merging *.v8.2
#Actually... not sure why we were doing this at all
#This is from *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# 🧼 Step 1: Clean merge inputs
valid_rows = df_anomalies_full["timestamp"].notna()
merge_subset = df_anomalies_full.loc[valid_rows].copy()
route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# 🔧 Ensure matching dtypes
merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

In [100]:
# Step 1: Clean anomaly type column
df_temp = df_anomalies_full.copy()
df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")
df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot = df_temp.pivot_table(
    index="route_short_name",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed"
    # , "disappearance", "early_appearance"
]
pivot = pivot.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot.columns.tolist())


✅ Total in pivot: 19398
✅ Total in df: 19398
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed']


In [101]:
pivot

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed
route_short_name,,,,,,
(Missing),0,0,13873,0,0,0
1,0,0,0,140,0,0
10,0,0,0,73,0,0
11,0,0,0,62,0,0
140,0,0,0,63,0,0
141,0,0,0,364,0,0
155,0,0,0,308,0,0
157,0,0,0,218,0,0
16,0,0,0,378,0,0


In [102]:
#Second pivot table to try looking at things other than route_short_name
# Step 1: Clean anomaly type column
df_temp2 = df_anomalies_full.copy()
df_temp2["trip_id"] = df_temp2["trip_id"].fillna("(Missing)")
df_temp2["anomaly_type_clean"] = df_temp2["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot2 = df_temp2.pivot_table(
    index="trip_id",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot2 = pivot2.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot2.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot2.columns.tolist())

✅ Total in pivot: 19398
✅ Total in df: 19398
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


In [103]:
pivot2

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
trip_id,,,,,,,,
600305,0,0,12,0,0,0,0,0
600306,0,0,11,0,0,0,0,0
600307,0,0,12,0,0,0,0,0
600308,0,0,11,0,0,0,0,0
600309,0,0,12,4,0,0,0,0
...,...,...,...,...,...,...,...,...
603198,0,0,10,3,0,0,0,0
603199,0,0,7,0,0,0,0,0
603200,0,0,9,0,0,0,0,0


#Plot in Folium

In [104]:
# Use full anomaly dataframe
df_full2 = df_anomalies_full.copy()

# Ensure lat/lon are floats
df_full2["latitude"] = pd.to_numeric(df_full2["latitude"], errors="coerce")
df_full2["longitude"] = pd.to_numeric(df_full2["longitude"], errors="coerce")

# Ensure timestamp_collected is datetime
df_full2["timestamp_collected"] = pd.to_datetime(df_full2["timestamp_collected"], errors="coerce", utc=True)

# Compute diffs for position and time
df_full2["lat_diff"] = df_full2.groupby("vehicle_id")["latitude"].diff()
df_full2["lon_diff"] = df_full2.groupby("vehicle_id")["longitude"].diff()
df_full2["jump_dist"] = (df_full2["lat_diff"]**2 + df_full2["lon_diff"]**2)**0.5
df_full2["time_diff"] = df_full2.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label jump and disappearance events using anomaly_params
df_full2["is_jump"] = df_full2["jump_dist"] > anomaly_params["JUMP_DISTANCE_THRESHOLD"]
df_full2["is_disappearance"] = df_full2["time_diff"] > anomaly_params["DISAPPEARANCE_TIME_THRESHOLD"]

# Create enriched jump DataFrame
jumps_df = df_full2[df_full2["is_jump"]].copy()
jumps_df["lat_prev"] = df_full2.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_full2.groupby("vehicle_id")["longitude"].shift()
jumps_df["timestamp_prev"] = df_full2.groupby("vehicle_id")["timestamp_collected"].shift()
jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# Filter to vehicles with enough jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= anomaly_params["MIN_JUMP_COUNT_PER_VEHICLE"]].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Base frame of jumpers for disappearance/reappearance
df_jumpers_only = df_full2[df_full2["vehicle_id"].isin(keep_jumpers)].copy()

#Depricated
# # Identify disappearance and reappearance points
# disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
# is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
# reappear_df = df_jumpers_only[is_reappear].copy()

# Summary of Key Changes:
# ✅ Replaces JUMP_DISTANCE_THRESHOLD, DISAPPEARANCE_TIME_THRESHOLD, and MIN_JUMP_COUNT_PER_VEHICLE with their anomaly_params equivalents.
# ✅ Maintains existing structure and logic.
# ✅ Compatible with all downstream Folium plotting layers.


In [105]:
#Step 1: Extract shape_id to route_id from trips_df
shape_to_route = trips[["shape_id", "route_id"]].drop_duplicates()
# trips.txt defines which route uses which shape.
# shape_to_route gives a clean 1:1 or 1:many mapping.

#Step 2: Map route_id to route_short_name from routes_df
route_id_to_name = routes[["route_id", "route_short_name"]]
#Brings in human-readable route names like "1", "10", "66".

#Step 3: Merge to associate shape_id with route_short_name
shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
#You now have a table with: shape_id, route_id, and route_short_name

shapes_copy = shapes.copy()

# Step 4: Drop preexisting route_short_name to avoid merge conflicts
if "route_short_name" in shapes_copy.columns:
    shapes_copy.drop(columns=["route_short_name"], inplace=True)

# Step 4 (retry): Merge into shapes_df
shapes_df = shapes_copy.merge(shape_route_map, on="shape_id", how="left")

# Step 5: Confirm structure
assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())





✅ Final columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


In [ ]:
# --- Initialize map ---
mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# --- Helpers ---
def format_route(route):
    return f"Route {str(route).zfill(3)}" if pd.notna(route) else "(Missing)"

def safe_route_key(val):
    try:
        return int(str(val))
    except:
        return float("inf")

# --- Define color map for anomalies ---
anomaly_colors = {
    "jump_or_gap": "orange",
    "stuck_vehicle": "red",
    "impossible_speed": "darkred",
    "backtracking": "blue",
    # "repeated_points": "purple",
    # "disappearance": "black",
    "early_appearance": "green",
    "off_route": "cadetblue",
    # "reappearance": "lime"
}

# --- Collect all unique route names across all datasets ---
all_routes = set()
for df in [df_anomalies_full, jumps_df, disappear_df, reappear_df, shapes_df]:
    if df is not None and not df.empty and "route_short_name" in df.columns:
        all_routes.update(df["route_short_name"].dropna().unique())

# --- Loop through each route ---
for route in sorted(all_routes, key=safe_route_key):
    route_label = format_route(route)

    # 1. --- Anomalies ---
    if not df_anomalies_full.empty:
        df_route = df_anomalies_full[df_anomalies_full["route_short_name"] == route]
        for anomaly_type in df_route["anomaly_type"].dropna().unique():
            color = anomaly_colors.get(anomaly_type, "gray")
            group = FeatureGroup(name=f"[{route_label}] {anomaly_type.replace('_', ' ').title()}", show=False)

            for _, row in df_route[df_route["anomaly_type"] == anomaly_type].iterrows():
                if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                    popup = (
                        f"Anomaly: {anomaly_type}<br>"
                        f"Vehicle: {row.get('vehicle_id')}<br>"
                        f"Trip: {row.get('trip_id')}<br>"
                        f"Time: {row.get('timestamp_collected')}"
                    )
                    CircleMarker(
                        location=[row["latitude"], row["longitude"]],
                        radius=4,
                        color=color,
                        fill=True,
                        fill_opacity=0.9,
                        popup=popup
                    ).add_to(group)
            group.add_to(mymap)

    # 2. --- Jump Lines and Points ---
    if "jumps_df" in globals() and not jumps_df.empty:
        df_jump = jumps_df[jumps_df["route_short_name"] == route]

        if not df_jump.empty:
            line_group = FeatureGroup(name=f"[{route_label}] Jump Lines", show=False)
            point_group = FeatureGroup(name=f"[{route_label}] Jump Start/End", show=False)

            for _, row in df_jump.iterrows():
                if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
                    start = [row["lat_prev"], row["lon_prev"]]
                    end = [row["latitude"], row["longitude"]]
                    vehicle = row.get("vehicle_id", "N/A")
                    t_prev = row.get("timestamp_prev", "N/A")
                    t_curr = row.get("timestamp_curr", "N/A")

                    PolyLine([start, end], color="orange", weight=2,
                             tooltip=f"Vehicle {vehicle} jump").add_to(line_group)

                    CircleMarker(location=start, radius=4, color="blue", fill=True,
                                 fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(point_group)
                    CircleMarker(location=end, radius=4, color="purple", fill=True,
                                 fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(point_group)

            line_group.add_to(mymap)
            point_group.add_to(mymap)

    # # 3. --- Disappearance / Reappearance ---
    # for df_layer, icon_color, label_suffix in [
    #     (disappear_df, "red", "Disappearances"),
    #     (reappear_df, "green", "Reappearances")
    # ]:
    #     if df_layer is not None and not df_layer.empty:
    #         df_sub = df_layer[df_layer["route_short_name"] == route]
    #         if not df_sub.empty:
    #             group = FeatureGroup(name=f"[{route_label}] {label_suffix}", show=False)
    #             for _, row in df_sub.iterrows():
    #                 if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
    #                     Marker(
    #                         location=[row["latitude"], row["longitude"]],
    #                         icon=Icon(color=icon_color, icon="info-sign"),
    #                         tooltip=f"Vehicle {row.get('vehicle_id')}<br>{row.get('timestamp_collected')}"
    #                     ).add_to(group)
    #             group.add_to(mymap)

    # 4. --- Route Shape ---
    if "shapes_df" in globals() and not shapes_df.empty:
        df_shapes = shapes_df[shapes_df["route_short_name"] == route]
        if not df_shapes.empty:
            group = FeatureGroup(name=f"[{route_label}] Route Shape", show=False)
            for shape_id, shape_data in df_shapes.groupby("shape_id"):
                shape_data = shape_data.sort_values("shape_pt_sequence")
                latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
                PolyLine(
                    locations=latlons,
                    color="gray",
                    weight=2,
                    opacity=0.6,
                    popup=f"{route_label} | Shape {shape_id}"
                ).add_to(group)
            group.add_to(mymap)

# --- Add control and save ---
LayerControl(collapsed=False).add_to(mymap)
mymap.save("anomaly_map_grouped_by_route_and_type.html")
print("✅ Map saved to 'anomaly_map_grouped_by_route_and_type.html'")
mymap
